In [2]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [3]:
documents=[]
f = open("default-data.txt", "r")
for x in f:
  documents.append(x)

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-01-23 17:04:30,081 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-01-23 17:04:30,082 : INFO : collecting all words and their counts
2019-01-23 17:04:30,083 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-01-23 17:04:30,085 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-23 17:04:30,178 : INFO : PROGRESS: at sentence #10000, processed 784348 words, keeping 28 word types
2019-01-23 17:04:30,238 : INFO : collected 46 word types from a corpus of 1304174 raw words and 16456 sentences
2019-01-23 17:04:30,239 : INFO : Loading a fresh vocabulary
2019-01-23 17:04:30,241 : INFO : effective_min_count=2 retains 46 unique words (100% of original 46, drops 0)
2019-01-23 17:04:30,242 : INFO : effective_min_count=2 leaves 1304174 word corpus (100% of original 1304174, drops 0)
2019-01-23 17:04:30,242 : INFO : deleting the raw count

2019-01-23 17:04:32,193 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-01-23 17:04:32,195 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-01-23 17:04:32,198 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-01-23 17:04:32,199 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-01-23 17:04:32,200 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-01-23 17:04:32,201 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-01-23 17:04:32,202 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-01-23 17:04:32,203 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-01-23 17:04:32,204 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-01-23 17:04:32,204 : INFO : EPOCH - 2 : training on 1304174 raw words (226205 effective words) took 0.2s, 917119 effective words/s
2019-01-23 17:04:32,493 : INFO : worker threa

2019-01-23 17:04:34,863 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-01-23 17:04:34,864 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-01-23 17:04:34,865 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-01-23 17:04:34,866 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-01-23 17:04:34,867 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-01-23 17:04:34,867 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-01-23 17:04:34,869 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-01-23 17:04:34,870 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-01-23 17:04:34,871 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-01-23 17:04:34,872 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-01-23 17:04:34,872 : INFO : EPOCH - 10 : training on 1304174 raw words (225922 effect

(2261584, 13041740)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [ ]:

w1 = "get"
model.wv.most_similar (positive=w1)


That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


In [ ]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


In [ ]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [ ]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [ ]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

In [ ]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

In [ ]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
